# **Documentation**
### Julia functions in the `eds-classification` repository
Developed by Austin M. Weber, Byrd Polar and Climate Research Center, Columbus, Ohio.

---
## `donarummo_classification.jl`
Mineral sorting scheme by Donarummo et al. (2003)

**Syntax**
```julia; term=true
donarummo_classification(data)
minerals = donarummo_classification(data)
```

**Inputs**
```julia; term=true
data --> Must be a DataFrame of EDS net intensities with a column for the following elements: Na, Mg, Al, Si, K, Ca, and Fe
```

**Outputs**
```julia; term=true
minerals --> List of mineral assignments for each row in the input data
```

**Example**

In [2]:
## Load dependencies
using Random,DataFrames
include("donarummo_classification.jl"); # Function file must be in the current folder

## Synthetic data
sz = (1000,1)
Na = rand(1:4000, sz)
Mg = rand(1:1000, sz)
Al = rand(1:12000, sz)
Si = rand(1:30000, sz)
K = rand(1:2000, sz)
Ca = rand(1:4000, sz)
Fe = rand(1:6000, sz)
elements = hcat(Na, Mg, Al, Si, K, Ca, Fe)
element_names = [:Sodium, :Mg, :ALUMinium, :silicon, :K, :Ca, :Fe]
data_table = DataFrame(elements, Symbol.(element_names))

## Use the Donarummo algorithm to classify the data
minerals = donarummo_classification(data_table);
first(minerals,5)

Row,Minerals
,String
1,U-C1
2,U-D5
3,U-E
4,U-B1
5,Hbl


**Note** The column names in `data_table` can be either the spelled-out name of the element or the element abbreviation. Capitalization does not matter.

**Additional details**
The Donarummo algorithm will assign a mineral name or unknown classification to each row in the input table. The full list of possible mineral assignments is given below using abbreviations from Whitney & Evans (2010):

Abbreviation | Mineral | Description
:-- | :-- | :--
Ab | Albite | Na-feldspar
An | Anorthite | Ca-feldspar
Aug | Augite | Pyroxene family
Bt | Biotite | Mica family
Chl | Chlorite | Clay mineral
Hbl | Hornblende | Amphibole family
Htr | Hectorite | Clay mineral
Ilt | Illite | Clay mineral
Ilt/Sme | Illite/Smectite | 70%/30% clay mixture
Kln | Kaolinite | Clay mineral
Lab/Byt | Labradorite/Bytownite | Ca>Na-feldspar
Mnt | Montmorillonite | Clay mineral
Ms | Muscovite | Mica family & clay mineral
Olig/Ans | Oligoclase/Andesine | Na>Ca-feldspar
Afs | Orthoclase | Alkali "K"-feldspar
Vrm | Vermiculite | Clay mineral
U-## | Unknown | Different "unknown" mineral branches


**References**
```tex; term=true
Donarummo et al. (2003). Geophyiscal Research Letters 30(6), 1269. https://doi.org/10.1029/2002GL016641

Whitney, D. L., & Evans, B. W. (2010). Abbreviations for names of  rock-forming minerals. American Mineralogist, 95(1), 185–187   https://doi.org/10.2138/am.2010.3371 
```

---
## `kandler_classification.jl`
Mineral identification algorithm by Kandler et al. (2011)

**Syntax**
```julia; term=true
kandler_classification(data)
classes = kandler_classification(data)
```

**Inputs**
```julia; term=true
data --> Must be a DataFrame of EDS net intensities with a column for the following elements: Na, Mg, Al, Si, P, S, Cl, K, Ca, Ti, Cr, Mn, and Fe
```

**Outputs**
```julia; term=true
classes --> List of mineral classes for each row in the input data
```

**Example**

In [2]:
## Load dependencies
using Random, DataFrames
include("kandler_classification.jl"); # Function file must be in the current folder

## Synthetic data
sz = (1000,1);
Na = rand(0:20, sz); Mg = rand(0:20, sz); Al = rand(10:70, sz); Si = rand(10:90, sz); 
P  = rand(0:1, sz); S  = rand(0:10, sz); Cl = rand(0:1, sz); K = rand(0:20, sz); Ca = rand(0:20, sz); 
Ti = rand(0:20, sz); Cr = rand(0:0.5, sz); Mn = rand(0:0.5, sz); Fe = rand(0:20, sz)
elements = hcat(Na, Mg, Al, Si, P, S, Cl, K, Ca, Ti, Cr, Mn, Fe)
element_names = [:Sodium, :Mg, :ALUMinium, :silicon, :P, :S, :Cl, :K, :Ca, :Ti, :Cr, :Mn, :Fe]
kandler_data_table = DataFrame(elements, Symbol.(element_names))
# Normalize each row to add to 100
function normalize_to_100!(df::DataFrame)
 data = Matrix(df)
 row_sums = sum(data, dims=2)
 normalization_factors = 100 ./ row_sums
 df .= data .* normalization_factors
 return df
end
kandler_data_table = normalize_to_100!(kandler_data_table)

## Use the Kandler algorithm to classify the data
classes = kandler_classification(kandler_data_table);
first(classes,5)

Row,Minerals
,String
1,SiAlNaCa
2,other Si-dominated
3,mixtures Ca+SiAl
4,other Si-dominated
5,other Si-dominated


**Note** The column names in `data_table` can be either the spelled-out name of the element or the element abbreviation. Capitalization does not matter.

**Additional details**
The Kandler algorithm will assign a mineral class to each row in the input table. The full list of possible mineral assignments is given below:

Number | Class | Description
:-- | :-- | :-- 
01 | biological | Biological (carbonaceous) material
02 | Na-rich | Na-rich mineral 
03 | ammonium sulfate | NH<sub>4</sub>SO<sub>4</sub> 
04 | Na sulfate | Na<sub>2</sub>SO<sub>4</sub> 
05 | Ca Na sulfate | (Na,Ca)<sub>2</sub>SO<sub>4</sub>
06 | Ca sulfate | CaSO<sub>4</sub>
07 | Other sulfate | Other sulfate mineral 
08 | Ca carbonate | CaCO<sub>3</sub> 
09 | Ca Mg carbonate | CaMg(CO<sub>3</sub>)<sub>2</sub> 
10 | phosphate | Phosphate mineral
11 | Na chloride | NaCl 
12 | K chloride | KCl 
13 | other chloride | Other chloride mineral 
14 | Fe oxide | Fe oxide mineral 
15 | Ti oxide | Ti oxide mineral 
16 | Fe Ti oxide | Fe+Ti oxide mineral 
17 | Al oxide | Aluminum oxide mineral
18 | quartz | SiO<sub>2</sub>
19 | SiAl | Aluminosilicate 
20 | SiAlK | K-bearing aluminosilicate 
21 | SiAlNa | Na-bearing aluminosilicate
22 | SiAlNaCa | Na- and Ca-bearing aluminosilicate
23 | SiAlNaK | Na- and K-bearing aluminosilicate
24 | SiAlCaFeMg | Ca-, Fe-, and Mg-bearing aluminosilicate
25 | SiAlKFeMg | K-, Fe-, and Mg-bearing aluminosilicate
26 | SiAlFeMg | Fe- and Mg-bearing aluminosilicate
27 | SiMgFe | Mg- and Fe-bearing silicate
28 | SiMg | Magnesiosilicate
29 | SiCaTi | Ca- and Ti-bearing silicate
30 | mixtures Si+S | Silicate and sulfate mixture
31 | mixtures SiAl+S | Aluminosilicate and sulfate mixture
32 | mixtures Cl+S | Chloride and sulfate mixture
33 | mixtures NaCl+SiAl | Sodium chloride and aluminosilicate mixture
34 | mixtures Ca+Si | Calcareous silicate mixture
35 | mixtures Ca+SiAl | Calcareous aluminosilicate mixture
36 | other Si-dominated | Other silica-dominated mixture
37 | steel | Steel
38 | other Mg-dominated | Other magnesium-dominated mineral
39 | other K-dominated | Other potassium-dominated mineral
40 | other Ca-dominated | Other calcium-dominated mineral

**References**
```tex; term=true
Kandler et al. (2011). Tellus B 63(4), 475–496. https://doi.org/10.1111/j.1600-0889.2011.00550.x
```

---
## `panta_classification.jl`
Mineral identification algorithm by Panta et al. (2023)

**Syntax**
```julia; term=true
panta_classification(data)
mineral = panta_classification(data)
```

**Inputs**
```julia; term=true
data --> Must be a DataFrame of EDS net intensities with a column for the following elements: F, Na, Mg, Al, Si, P, S, Cl, K, Ca, Ti, Cr, Mn, and Fe
```

**Outputs**
```julia; term=true
mineral --> List of mineral assignments for each row in the input data
```

**Example**

In [15]:
## Load dependencies
using Random, DataFrames
include("panta_classification.jl"); # Function file must be in the current folder

## Synthetic data
rng = Random.Xoshiro(56)
sz = (1000,1);
F = rand(rng, 0:0.5, sz); Na = rand(rng, 0:20, sz); Mg = rand(rng, 0:20, sz); Al = rand(rng, 10:70, sz); Si = rand(rng, 10:90, sz); 
P  = rand(rng, 0:1, sz); S  = rand(rng, 0:10, sz); Cl = rand(rng, 0:1, sz); K = rand(rng, 0:20, sz); Ca = rand(rng, 0:20, sz); 
Ti = rand(rng, 0:20, sz); Cr = rand(rng, 0:0.5, sz); Mn = rand(rng, 0:0.5, sz); Fe = rand(rng, 0:20, sz)
elements = hcat(F,Na, Mg, Al, Si, P, S, Cl, K, Ca, Ti, Cr, Mn, Fe)
element_names = [:F, :Sodium, :Mg, :ALUMinium, :silicon, :P, :S, :Cl, :K, :Ca, :Ti, :Cr, :Mn, :Fe]
panta_data_table = DataFrame(elements, Symbol.(element_names));
# Normalize rows to add up to 100
panta_data_table = normalize_to_100!(panta_data_table);

## Use the Panta algorithm to classify the data
panta_minerals = panta_classification(panta_data_table);
first(panta_minerals,5)

Row,Minerals
,String
1,Illite-like
2,Kaolinite-like
3,Unknown
4,Kaolinite-like
5,Unknown


**Note** The column names in `data_table` can be either the spelled-out name of the element or the element abbreviation. Capitalization does not matter. Also, in this case, because the data was completely synthetic, some of the classifications are assigned "Unknown". This will happen if none of the values can fit properly within the constraints of the mineral compositional ranges.

**Additional details**
The Panta algorithm will assign a mineral name to each row in the input table. The full list of possible mineral assignments is given below:

Number | Mineral 
:-- | :-- 
01 | Albite-like
02 | Alunite-like
03 | Apatite-like
04 | Calcite-like
05 | Chlorite-like
06 | Dolomite-like
07 | Feldspar-like
08 | Gypsum-like
09 | Halite-like
10 | Hematite-like
11 | Ilmenite-like
12 | Illite-like
13 | Kaolinite-like
14 | Mica-like
15 | Microcline-like
16 | Quartz-like
17 | Rutile-like
18 | Smectite-like
19 | Ca-rich silicate/Ca-Si-mix
20 | Complex clay
21 | Complex feldspar
22 | Complex quartz
23 | Complex sulfate


**References**
```tex; term=true
Panta et al. (2023). Atmospheric Chemistry and Physics 23, 3861-3885. https://doi.org/10.5194/acp-23-3861-2023

```